# Demo with a vlm dataset (openbmb/RLAIF-V-Dataset)

In [1]:
!pip install -q kagglehub

!pip install -q tensorflow
!pip install -q tensorboardX
!pip install -q grain
# !pip install -q git+https://github.com/google/tunix
!pip install -q git+https://github.com/google/qwix

!pip uninstall -q -y flax
!pip install -q git+https://github.com/google/flax.git

!pip install -q huggingface_hub
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 136.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 136.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 150.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wh

In [2]:
%cd /content
!git clone --branch uiuc-vlm --single-branch https://github.com/PLAN-Lab/tunix.git


/content
Cloning into 'tunix'...
remote: Enumerating objects: 2393, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 2393 (delta 104), reused 71 (delta 69), pack-reused 2222 (from 3)
Receiving objects: 100% (2393/2393), 5.80 MiB | 23.93 MiB/s, done.
Resolving deltas: 100% (1448/1448), done.


In [3]:
%cd /content/tunix
!pip -q install -e .

/content/tunix
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.6 MB/s eta 0:00:00
  Building editable for tunix (pyproject.toml) ... done


In [4]:
# --- autoreload for dev loop ---
import sys, types, importlib
if 'imp' not in sys.modules:
    imp = types.ModuleType('imp')
    imp.reload = importlib.reload
    sys.modules['imp'] = imp

%load_ext autoreload
%autoreload 2

# sanity check import
import tunix
print("tunix imported from:", tunix.__file__)

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


tunix imported from: /content/tunix/tunix/__init__.py


In [5]:
import functools
import gc
import os
from pprint import pprint
import re
import time

from flax import nnx
import grain
import humanize
import jax
import jax.numpy as jnp
import kagglehub
import optax
from orbax import checkpoint as ocp
import qwix
import tensorflow_datasets as tfds
from tqdm.auto import tqdm

from tunix.examples.data import translation_dataset as data_lib
from tunix.generate import sampler as sampler_lib
from tunix.generate.vlm_sampler import VLMSampler
from tunix.models.gemma3 import params as params_lib
from tunix.models.gemma3 import params_safetensors as params_safetensors_lib

from tunix.sft import metrics_logger
from datasets import load_dataset
from tunix.sft.dpo.dpo_trainer import DpoTrainingConfig
from tunix.sft.dpo.dpo_trainer import DpoTrainer
from tunix.sft.dpo.dpo_trainer import TrainingInput
from huggingface_hub import snapshot_download
from tunix.sft.dpo.dpo_trainer import _generate_ids_and_masks
from tunix.models.gemma3 import model as gemma3_model_lib
from datasets import concatenate_datasets

In [6]:
# ====== Data ======
TRAIN_DATA_DIR = "./data/train"
TEST_DATA_DIR = "./data/test"
TRAIN_FRACTION = 1.0

INTERMEDIATE_CKPT_DIR = "/content/intermediate_ckpt/"
# ====== LoRA ======
RANK = 8
ALPHA = 16.0

# ====== Sharding ======
MESH = [(1, 1), ("fsdp", "tp")]

MAX_PROMPT_LENGTH = 192
TOTAL_GENERATION_STEPS = 192
TEMPERATURE = 0.7
TOP_P = 1.0
TOP_K = 50
BETA = 0.1

# === AdamW, warmup, cosine scheduler ===
LEARNING_RATE = 3e-6
B1 = 0.9
B2 = 0.99
WEIGHT_DECAY = 0.1

# == Cosine decay with warmup scheduler ==
# Linearly increase learning rate from 0. to 5e-6 in the first 10% training
# steps, and then gradually decrease the learning rate to 0 using cosine
# scheduler.
BATCH_SIZE = 1
NUM_BATCHES = 512
NUM_TEST_BATCHES = 100
EVAL_EVERY_N_STEPS = 100

NUM_EPOCHS = 1  # can potentially train for more epochs
TRAIN_FRACTION = 1.0
MAX_STEPS = int(NUM_BATCHES * TRAIN_FRACTION * NUM_EPOCHS)

WARMUP_STEPS = 0.1 * MAX_STEPS
# == Grad clipping ==
# Grad clipping to prevent large gradients. Found this
# important to keep KL divergence in check.
MAX_GRAD_NORM = 0.1

# ====== Inference ======
GENERATION_CONFIGS = {
    # greedy search
    "greedy": {"temperature": 1e-4, "top_k": 1, "top_p": 1.0},
    # some randomness
    "standard": {"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    # liberal
    "liberal": {"temperature": 0.85, "top_k": 2000, "top_p": 1.0},
}

In [ ]:

from huggingface_hub import login, HfApi
import os

HF_TOKEN = "xxxxxxxxxxx"  # <-- paste your Read token



In [8]:
model_id = "google/gemma-3-1b-it"
ignore_patterns = [
    "*.pth",  # Ignore PyTorch .pth weight files
]
print(f"Downloading {model_id} from Hugging Face...")
local_model_path = snapshot_download(
    repo_id=model_id, ignore_patterns=ignore_patterns, token=HF_TOKEN
)
print(f"Model successfully downloaded to: {local_model_path}")

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/24.3k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Model successfully downloaded to: /root/.cache/huggingface/hub/models--google--gemma-3-1b-it/snapshots/dcc83ea841ab6100d6b47a070329e1ba4cf78752


In [9]:
MODEL_CP_PATH = local_model_path

model_config = (
    gemma3_model_lib.Gemma3Config.gemma3_1b()
)  # pick correponding config based on model version
MESH = [(1, 1), ("fsdp", "tp")]
mesh = jax.make_mesh(*MESH)
with mesh:
  gemma3 = params_safetensors_lib.create_model_from_safe_tensors(
      MODEL_CP_PATH, model_config, mesh
  )
  nnx.display(gemma3)

In [10]:
# ==== 3) Load only the tokenizer first (cheap), NOT the whole processor yet ====
from transformers import AutoTokenizer
tok = AutoTokenizer.from_pretrained("google/paligemma-3b-mix-224", token=HF_TOKEN)
print("Tokenizer loaded. pad_id:", tok.pad_token_id, "eos_id:", tok.eos_token_id)

# Adapter that your sampler expects
class HFTokenizerAdapter:
    def __init__(self, hf_tok):
        self.tok = hf_tok
        self._pad_id = self.tok.pad_token_id if self.tok.pad_token_id is not None else self.tok.eos_token_id
        self._eos_id = self.tok.eos_token_id
    def encode(self, s: str):
        return self.tok(s, add_special_tokens=False)["input_ids"]
    def decode(self, ids):
        return self.tok.decode(ids, skip_special_tokens=True)
    def pad_id(self) -> int: return int(self._pad_id)
    def eos_id(self) -> int: return int(self._eos_id)

gemma_tokenizer = HFTokenizerAdapter(tok)
print("Tokenizer adapter ready ✔️")

image_size = 224
print("Processor loaded. image_size:", image_size)

# ==== 5) Construct the sampler LAST (avoid touching model state in __init__) ====
# IMPORTANT: gemma3 must be your PaLI-Gemma nnx Module (not a text-only Gemma).
# If it's named differently, change transformer=... accordingly.
vlm_sampler = VLMSampler(
    transformer=gemma3,
    tokenizer=gemma_tokenizer,
    image_size=image_size,
)
print("VLMSampler ready. pad_id:", vlm_sampler.pad_id(), "eos_id:", vlm_sampler.eos_id())

# --- (Optional) 10-second smoke test with a dummy image ---
# import numpy as np, jax.numpy as jnp
# dummy = np.zeros((1, image_size, image_size, 3), dtype=np.uint8)
# out = vlm_sampler(
#     input_strings=["Describe the image:"],
#     images=jnp.asarray(dummy),
#     max_generation_steps=8,
#     temperature=0.0,
#     return_logits=False,
#     echo=False,
# )
# print(out.text[0])

/usr/local/lib/python3.12/dist-packages/torch_xla/experimental/gru.py:113: SyntaxWarning: invalid escape sequence '\_'
  * **h_n**: tensor of shape :math:`(D * \text{num\_layers}, H_{out})` or
/usr/local/lib/python3.12/dist-packages/torch_xla/__init__.py:258: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

Tokenizer loaded. pad_id: 0 eos_id: 1
Tokenizer adapter ready ✔️
Processor loaded. image_size: 224
VLMSampler ready. pad_id: 0 eos_id: 1


In [11]:
def get_lora_model(base_model, mesh):
  lora_provider = qwix.LoraProvider(
      module_path=(
          ".*q_einsum|.*kv_einsum|.*gate_proj|.*down_proj|.*up_proj|"
          ".*attn_vec_einsum"
      ),
      rank=RANK,
      alpha=ALPHA,
      #weight_qtype="nf4",
      #tile_size=4,
  )

  model_input = base_model.get_model_input()
  lora_model = qwix.apply_lora_to_model(
      base_model, lora_provider, **model_input
  )

  with mesh:
    state = nnx.state(lora_model)
    pspecs = nnx.get_partition_spec(state)
    sharded_state = jax.lax.with_sharding_constraint(state, pspecs)
    nnx.update(lora_model, sharded_state)

  return lora_model

In [12]:
# Policy model
lora_gemma = get_lora_model(gemma3, mesh=mesh)
nnx.display(lora_gemma)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from datasets import load_dataset
import numpy as np, jax.numpy as jnp
from PIL import Image
from tunix.models.siglip import preprocess as siglip_pp
from transformers import AutoTokenizer # Import AutoTokenizer

# ==== 3) Load only the tokenizer first (cheap), NOT the whole processor yet ====
# Assuming gemma_tokenizer is already defined and is an instance of HFTokenizerAdapter
# If not, you would need to load it here or in a previous cell.
# from transformers import AutoTokenizer
# tok = AutoTokenizer.from_pretrained("google/paligemma-3b-mix-224", token=HF_TOKEN)
# class HFTokenizerAdapter: ... # Define the adapter if not already defined
# gemma_tokenizer = HFTokenizerAdapter(tok)


SPLIT = "train[:64]"
MAX_PROMPT_LEN = 128
IMAGE_SIZE = 224

ds = load_dataset("openbmb/RLAIF-V-Dataset", split=SPLIT, token=HF_TOKEN)
cols = ["image", "question", "chosen", "rejected"]
ds = ds.remove_columns([c for c in ds.column_names if c not in cols])

def _pick_one_image(img_field):
    """Return a single PIL.Image from the dataset's image field."""
    x = img_field
    if isinstance(x, list):
        if not x:  # empty list, skip later
            return None
        x = x[0]
    if isinstance(x, Image.Image):
        return x.convert("RGB")
    # HF 'Image' feature sometimes gives np.ndarray
    arr = np.array(x)
    if arr.ndim == 3:
        return Image.fromarray(arr).convert("RGB")
    return None

def preprocess_item(ex):
    img = _pick_one_image(ex["image"])
    if img is None:
        return {
            "pixel_values": None,
            "question": ex["question"],
            "chosen": ex["chosen"],
            "rejected": ex["rejected"],
        }
    arr = np.array(img, dtype=np.uint8)[None, ...]              # [1,H,W,3]
    px  = siglip_pp.preprocess(jnp.asarray(arr), IMAGE_SIZE)     # [1,S,S,3] float32
    return {
        "pixel_values": np.asarray(px[0]),                       # [S,S,3]
        "question": ex["question"],
        "chosen": ex["chosen"],
        "rejected": ex["rejected"],
    }

ds = ds.with_transform(preprocess_item)

def numpy_batches(dataset, batch_size=4, shuffle=True, seed=0):
    idx = np.arange(len(dataset))
    if shuffle:
        rng = np.random.default_rng(seed)
        rng.shuffle(idx)
    buf = []
    for i in idx:
        ex = dataset[int(i)]
        # skip examples where we couldn't pick an image
        if ex["pixel_values"] is None:
            continue
        buf.append(ex)
        if len(buf) == batch_size:
            # Tokenize text fields individually and convert to JAX arrays
            questions_tokens = [gemma_tokenizer.tok(q, truncation=True, max_length=MAX_PROMPT_LEN)["input_ids"] for q in [b["question"] for b in buf]]
            chosen_tokens = [gemma_tokenizer.tok(c, truncation=True, max_length=MAX_PROMPT_LEN)["input_ids"] for c in [b["chosen"] for b in buf]]
            rejected_tokens = [gemma_tokenizer.tok(r, truncation=True, max_length=MAX_PROMPT_LEN)["input_ids"] for r in [b["rejected"] for b in buf]]

            # Pad and stack token IDs
            max_q_len = max(len(ids) for ids in questions_tokens)
            max_c_len = max(len(ids) for ids in chosen_tokens)
            max_r_len = max(len(ids) for ids in rejected_tokens)

            padded_questions = jnp.stack([jnp.pad(jnp.asarray(ids), (0, max_q_len - len(ids)), constant_values=gemma_tokenizer.pad_id()) for ids in questions_tokens], axis=0)
            padded_chosen = jnp.stack([jnp.pad(jnp.asarray(ids), (0, max_c_len - len(ids)), constant_values=gemma_tokenizer.pad_id()) for ids in chosen_tokens], axis=0)
            padded_rejected = jnp.stack([jnp.pad(jnp.asarray(ids), (0, max_r_len - len(ids)), constant_values=gemma_tokenizer.pad_id()) for ids in rejected_tokens], axis=0)


            yield {
                "pixel_values": jnp.asarray(np.stack([b["pixel_values"] for b in buf], axis=0)),
                "question":     padded_questions,
                "chosen":       padded_chosen,
                "rejected":     padded_rejected,
            }
            buf = []
    if buf:
        # Tokenize text fields individually and convert to JAX arrays for the last partial batch
        questions_tokens = [gemma_tokenizer.tok(q, truncation=True, max_length=MAX_PROMPT_LEN)["input_ids"] for q in [b["question"] for b in buf]]
        chosen_tokens = [gemma_tokenizer.tok(c, truncation=True, max_length=MAX_PROMPT_LEN)["input_ids"] for c in [b["chosen"] for b in buf]]
        rejected_tokens = [gemma_tokenizer.tok(r, truncation=True, max_length=MAX_PROMPT_LEN)["input_ids"] for r in [b["rejected"] for b in buf]]

        # Pad and stack token IDs
        max_q_len = max(len(ids) for ids in questions_tokens)
        max_c_len = max(len(ids) for ids in chosen_tokens)
        max_r_len = max(len(ids) for ids in rejected_tokens)

        padded_questions = jnp.stack([jnp.pad(jnp.asarray(ids), (0, max_q_len - len(ids)), constant_values=gemma_tokenizer.pad_id()) for ids in questions_tokens], axis=0)
        padded_chosen = jnp.stack([jnp.pad(jnp.asarray(ids), (0, max_c_len - len(ids)), constant_values=gemma_tokenizer.pad_id()) for ids in chosen_tokens], axis=0)
        padded_rejected = jnp.stack([jnp.pad(jnp.asarray(ids), (0, max_r_len - len(ids)), constant_values=gemma_tokenizer.pad_id()) for ids in rejected_tokens], axis=0)

        yield {
            "pixel_values": jnp.asarray(np.stack([b["pixel_values"] for b in buf], axis=0)),
            "question":     padded_questions,
            "chosen":       padded_chosen,
            "rejected":     padded_rejected,
        }


# Smoke one batch
b0 = next(numpy_batches(ds, batch_size=2))
print("Batch pixels:", b0["pixel_values"].shape, "| B:", len(b0["question"]))
print("Batch question tokens:", b0["question"].shape)
print("Batch chosen tokens:", b0["chosen"].shape)
print("Batch rejected tokens:", b0["rejected"].shape)

Batch pixels: (2, 224, 3) | B: 2
Batch question tokens: (2, 12)
Batch chosen tokens: (2, 128)
Batch rejected tokens: (2, 102)
